In [24]:
%pip install semantic-router==0.0.37
%pip install chromadb
%pip install pypdf
%pip install replicate

Python(2053) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


Python(2055) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


Python(2057) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


Python(2058) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ["COHERE_API_KEY"] = "kUYLefL4SNo36iNh5tF2H1gGi5XfZyTbOBJNknt1"

In [4]:
import chromadb
import os
client = chromadb.PersistentClient(
    path="/Users/anavbo/Desktop/Personal/skin-cancer-detection-app/RAG/chromadb",
)
collection = client.get_or_create_collection(name="skin-cancer")

# Fix Chroma to Cohere Encoder

In [5]:
import chromadb.utils.embedding_functions as embedding_functions
chroma_encode = embedding_functions.CohereEmbeddingFunction(
    api_key=os.environ["COHERE_API_KEY"], model_name="embed-english-v3.0"
)

In [6]:
# semantic chunking
from semantic_router.splitters import RollingWindowSplitter
from semantic_router.utils.logger import logger
from semantic_router.encoders import CohereEncoder

logger.setLevel("WARNING")
encoder = CohereEncoder()
splitter = RollingWindowSplitter(
    encoder=encoder,
    min_split_tokens=50,
    max_split_tokens=400,
    window_size=2,
    enable_statistics=False,
)

In [7]:
from pypdf import PdfReader
import os
import uuid
path="/Users/anavbo/Desktop/Personal/skin-cancer-detection-app/RAG/documents"
pdf_paths = os.listdir(path)

chunks = []
metadatas = []
ids = []
for doc_path in pdf_paths:
    documents = PdfReader(f"{path}/{doc_path}")
    document = ""
    for doc in documents.pages:
        document += doc.extract_text() + "\n"

    splits = splitter([document])
    for i in range(len(splits)):
        chunk = " ".join(splits[i].docs)
        try:
            pre_chunk = " ".join(splits[i - 1].docs)
        except:
            pre_chunk = ""
        try:
            post_chunk = " ".join(splits[i + 1].docs)
        except:
            post_chunk = ""
        
        metadata = {
            "source": doc_path,
            "pre_chunk": pre_chunk,
            "chunk": chunk,
            "post_chunk": post_chunk
        }
        chunks.append(chunk)
        metadatas.append(metadata)
        ids.append(str(uuid.uuid4().hex))

In [12]:
ids[0]

'a66975655a1a4a9898b93e5904806ace'

In [13]:
collection.add(
    documents = chunks,
    ids = ids,
    metadatas = metadatas,
)

Add of existing embedding ID: a66975655a1a4a9898b93e5904806ace
Insert of existing embedding ID: a66975655a1a4a9898b93e5904806ace


In [17]:
import chromadb

client = chromadb.PersistentClient(
    path="/Users/anavbo/Desktop/Personal/skin-cancer-detection-app/RAG/chromadb"
)
collection = client.get_collection("skin-cancer")



['According to this generally accepted model, eventually all intraepidermal proliferation of melanocytes ceases, and the nevus becomes entirely intradermal (dermal nevus). Nevus cells residing within the dermis have reduced proliferative and me tabolic activity, except for the formation of mel - anosomes. With the decline of replication, the nevus cell population is gradually replaced by mesenchymal elements, including fibrous matrix, glycosaminoglycans, and adipose tissue. Most dermal nevi are believed to undergo progressive involution, some eventuating as acrochordons, and others shedding. This is reflected in the course of the disease, as with advancing age, there is a progressive decrease in the number of nevi; thus, whereas the number of nevi in young adults varies from approximately 15 to 40, this decreases markedly over the age of 50 years.21 Clinical features Melanocytic nevi are well circumscribed, round to ovoid lesions, with regular and well defined borders. Although most ar

In [26]:
import replicate

# Get query from user
query = "What is actinic kerosis"


# Initialize collection for db requests
client = chromadb.PersistentClient(
    path="/Users/anavbo/Desktop/Personal/skin-cancer-detection-app/RAG/chromadb"
)
collection = client.get_collection("skin-cancer")

# Initialize the LLM
api = replicate.Client(api_token="r8_XJns3nBIXao9KgeqbdqSvgCTP8JDf1u18BiTx")

 Actinic keratosis (AK) is a precancerous condition that affects the skin. It is caused by prolonged exposure to the sun or other forms of ultraviolet (UV) radiation, and it can develop into squamous cell carcinoma, a type of skin cancer, if left untreated.

Actinic keratosis typically appears as a rough, scaly, or crusty bump on the skin, and it can be tan, pink, red, or the same color as the surrounding skin. It can appear anywhere on the body, but it is most common on sun-exposed areas such as the face, ears, neck, hands, and arms.

The signs and symptoms of actinic keratosis can vary, but they may include:

* A round or oval-shaped bump that is rough or waxy to the touch
* A flat growth or a slightly raised bump with a scaly surface
* A varied size, ranging from very small to over 1 inch (2.5 centimeters) across
* A varied number, ranging from a single growth to multiple growths
* Very small growths clustered around the eyes or elsewhere on the face, sometimes called "flesh moles" 

In [29]:
# Query the collection and form the context
context = ""
retrieval = collection.query(query_texts="actinic kerosis", n_results=3)
for i in range(len(retrieval["documents"][0])):
    context += f"Document {i}: {retrieval["documents"][0][i]}\n"

# Prompt the LLM
system_prompt = "Use the following context to craft the best answer: " + context
output = api.run(
    "meta/llama-2-70b-chat",
    input={
        "top_k": 0,
        "top_p": 1,
        "prompt": query,
        "max_tokens": 512,
        "temperature": 0.5,
        "system_prompt": system_prompt,
        "length_penalty": 1,
        "max_new_tokens": 500,
        "min_new_tokens": -1,
        "prompt_template": "<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{prompt} [/INST]",
        "presence_penalty": 0,
        "log_performance_metrics": False
    }
)
print("".join(output))

ReplicateError: ReplicateError Details:
title: Free time limit reached
status: 402
detail: You have reached the free time limit. To continue using Replicate, set up billing at https://replicate.com/account/billing#billing.